In [1]:
from timeit import default_timer as timer

In [2]:
!wget -N https://files.rcsb.org/download/1TGH.pdb
!wget -N https://files.rcsb.org/download/1CDW.pdb

--2020-04-20 16:23:03--  https://files.rcsb.org/download/1TGH.pdb
Resolving files.rcsb.org (files.rcsb.org)... 132.249.213.91
Connecting to files.rcsb.org (files.rcsb.org)|132.249.213.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/octet-stream]
Saving to: ‘1TGH.pdb.8’

1TGH.pdb.8              [ <=>                ] 215.95K  --.-KB/s    in 0.1s    

2020-04-20 16:23:03 (1.41 MB/s) - ‘1TGH.pdb.8’ saved [221130]

--2020-04-20 16:23:03--  https://files.rcsb.org/download/1CDW.pdb
Resolving files.rcsb.org (files.rcsb.org)... 132.249.213.91
Connecting to files.rcsb.org (files.rcsb.org)|132.249.213.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/octet-stream]
Saving to: ‘1CDW.pdb.7’

1CDW.pdb.7              [ <=>                ] 216.50K  --.-KB/s    in 0.1s    

2020-04-20 16:23:03 (1.47 MB/s) - ‘1CDW.pdb.7’ saved [221697]



In [3]:
#interactive 
import pytraj as pt
import nglview as nv

traj = pt.load('1TGH.pdb')
pt.rotate(traj, 'y 90')
view = nv.show_pytraj(traj)
view

NGLWidget()

In [4]:
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from sys import stdout
pdb = PDBFile('1TGH.pdb')
pdb.getTopology()

<Topology; 6 chains, 222 residues, 2355 atoms, 2421 bonds>

In [5]:
from pdbfixer import PDBFixer
fixer = PDBFixer(filename='1TGH.pdb')
fixer.findMissingResidues()
fixer.findNonstandardResidues()
fixer.replaceNonstandardResidues()
fixer.removeHeterogens(True)
fixer.findMissingAtoms()
fixer.addMissingAtoms()
fixer.addMissingHydrogens(7.0)
fixer.addSolvent(fixer.topology.getUnitCellDimensions())
PDBFile.writeFile(fixer.topology, fixer.positions, open('1TGH_fixed.pdb', 'w'))

In [6]:
fixer.topology

<Topology; 8 chains, 11117 residues, 36496 atoms, 25672 bonds>

In [7]:
traj = pt.load('1TGH_fixed.pdb')
pt.rotate(traj, 'y 90')
view 

NGLWidget(n_components=1)

In [19]:
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
#system = forcefield.createSystem(fixer.topology)
system = forcefield.createSystem(fixer.topology, nonbondedMethod=PME, nonbondedCutoff=1*nanometer, constraints=HBonds)
simulation = Simulation(fixer.topology, system, integrator)
simulation.context.setPositions(fixer.positions)
start = timer()
simulation.minimizeEnergy()
timer()-start

92.04592096900012

In [22]:
simulation.reporters.append(PDBReporter('1TGH_fixed_md.pdb',100))
simulation.reporters.append(StateDataReporter(stdout, 100, step=True,potentialEnergy=True, temperature=True))
simulation.step(10000)

1010,-590894.766835629,256.3978880349268
1020,-590902.8515498106,258.6879193394901
1030,-590730.1941501955,259.7368087808602
1040,-589653.7303771294,257.49800499022996
1050,-589715.8825121458,258.8828503143161
1060,-589322.6349433943,259.21176622033926
1070,-589375.6189666961,260.29556190706853
1080,-588851.8831833134,260.3641632655607
1090,-588509.6337682841,260.4102091639201
1100,-588715.7212898957,262.52220019409094
#"Step","Potential Energy (kJ/mole)","Temperature (K)"
1100,-588715.7212898957,262.52220019409094
1110,-589059.3449864294,264.93441235722577
1120,-588640.9351043081,265.5049031462283
1130,-587454.7907412968,263.8291206418332
1140,-587456.4986730781,264.6720212064828
1150,-587152.3016435794,265.0859806906455
1160,-586710.1780445244,265.2460921106552
1170,-586722.4408975412,267.2032560675225
1180,-586085.4573110515,266.2584812697372
1190,-586444.7778738331,268.9980630428905
1200,-585403.831619777,267.5166890966255
1200,-585403.831619777,267.5166890966255
1210,-585443.46236

2770,-572750.592922646,298.4959719440697
2780,-572505.1499725315,297.8529146717975
2790,-572477.0116484427,297.93720940992193
2800,-572576.3858063362,298.68797110631573
2800,-572576.3858063362,298.68797110631573
2810,-572696.9869976131,299.5459472210233
2820,-572366.4469679546,298.39642792196673
2830,-571970.3858173455,296.8417842087836
2840,-571831.9743024599,296.9465295679899
2850,-571871.1855005815,296.56285980015497
2860,-571435.8170827739,295.142225636386
2870,-571428.1822557396,294.59217487744473
2880,-571562.9756272539,294.2495975587051
2890,-571758.6748360632,294.85046991516384
2900,-572068.4580253871,295.84392153462295
2900,-572068.4580253871,295.84392153462295
2910,-572261.9270981953,295.9286797593602
2920,-572944.6880751603,298.4618413365007
2930,-572493.8858799646,296.15722682714494
2940,-572205.4826525318,296.164378848578
2950,-572077.3045051987,295.41942608139414
2960,-571681.013333188,294.89458734431327
2970,-572151.0590331759,296.7143338045044
2980,-571911.8907873025,29

4530,-570027.8960670066,298.5898020667023
4540,-570716.2320785794,300.5495892289639
4550,-570467.083664975,300.0983472590796
4560,-569910.7142713385,298.3587493593251
4570,-570149.3919713043,299.2214572908227
4580,-570104.4768466826,298.4870143913783
4590,-570279.4694768261,299.0410958620038
4600,-569873.8042737182,297.8457436569037
4600,-569873.8042737182,297.8457436569037
4610,-570180.8687587335,298.2688846534821
4620,-570529.9915720761,299.1446255287665
4630,-570103.1544799497,298.22955389408907
4640,-569800.6590563789,298.1130959576139
4650,-569971.4393119235,298.39594021591466
4660,-570185.899758363,298.8893638642588
4670,-569588.5339537214,296.9249725356183
4680,-570267.3489991844,299.1598958492344
4690,-570351.616116151,300.33119901047894
4700,-570054.191510925,299.5587267614455
4700,-570054.191510925,299.5587267614455
4710,-569798.7684653043,299.2810798192472
4720,-570448.4661117198,301.0446061077642
4730,-569779.465681465,298.5154359059838
4740,-569530.2227510932,297.914610224

6300,-569282.8339225927,302.1287929089069
6310,-568249.3388403971,298.53086352052014
6320,-568331.2887061728,297.7555288671992
6330,-569008.3025904325,300.39593989432285
6340,-569085.538956859,300.5517102329214
6350,-568581.4813156454,299.694836316159
6360,-569004.916617911,300.867821891965
6370,-568666.9666932548,299.2945138464008
6380,-568886.6769771854,299.91560095352406
6390,-568858.9919754685,299.86830935078166
6400,-569625.9915676971,302.1646317576115
6400,-569625.9915676971,302.1646317576115
6410,-569084.4281404123,300.5525509656255
6420,-569058.4948594299,300.5153315308331
6430,-568811.2971953835,300.05751115238365
6440,-569018.6521612272,300.3695990184683
6450,-569324.4575610844,300.7202788661094
6460,-568744.8989434128,298.9709525504263
6470,-569000.2095239338,300.80215818701237
6480,-569360.6121039446,301.7591923195627
6490,-568957.1325375306,300.7109967145703
6500,-568651.9133208555,300.23342045288507
6500,-568651.9133208555,300.23342045288507
6510,-569218.2873824452,301.87

8080,-569769.153959207,301.70973886413185
8090,-569765.7530413088,301.92106883116986
8100,-569346.165662299,300.47439982658057
8100,-569346.165662299,300.47439982658057
8110,-569327.8608575567,300.0470696620719
8120,-570388.7517031295,303.57721920434005
8130,-569360.0277331427,299.72191071363534
8140,-569211.0435391794,299.24390689359456
8150,-569676.8967950591,300.79480230016867
8160,-569640.154161749,299.7715742130019
8170,-569762.452959797,299.7889148400686
8180,-569822.6717432818,300.2064114358648
8190,-570034.5514421365,301.3986883341154
8200,-569499.3968539305,299.8560372100288
8200,-569499.3968539305,299.8560372100288
8210,-569913.9139159367,301.45800317583587
8220,-570013.67147155,301.4831562526344
8230,-570012.2712267864,301.6576371871375
8240,-570263.5390430213,302.3047932376883
8250,-570288.0267396782,301.9057924616066
8260,-570215.0310443591,301.5124397158721
8270,-569678.2187836058,299.83855111717713
8280,-569513.7827623056,299.26536078250047
8290,-570102.0983814509,300.05

9840,-570055.4912042979,304.0763312723999
9850,-569616.3055053744,302.157636106803
9860,-569076.9541623985,301.36967733232495
9870,-569862.2015851766,303.63412686917803
9880,-569621.237708355,302.44584506585045
9890,-569141.0162007695,301.66985081010995
9900,-569477.908837271,302.8536679682578
9900,-569477.908837271,302.8536679682578
9910,-569290.8771410108,301.8890056722805
9920,-569053.3222417232,301.0284786458893
9930,-569373.2924213995,302.66122542395226
9940,-570191.1730373311,305.4255069332786
9950,-570449.1568150142,305.8756572558574
9960,-569095.5651580486,301.30712771627776
9970,-568948.6797373431,300.9117652598018
9980,-569490.9115261103,303.01806001164135
9990,-569113.716459482,302.6863387352902
10000,-568925.9959958122,302.36055170461145
10000,-568925.9959958122,302.36055170461145
10010,-568427.5536197592,301.12697800320234
10020,-569104.1980782866,302.4774703998763
10030,-569046.9431381422,301.814562881939
10040,-569204.0975454198,301.9018436859255
10050,-569556.7392438157

In [ ]:
traj = pt.load('1TGH_fixed_md.pdb')
pt.rotate(traj, 'y 90')
view = nv.show_pytraj(traj)
#view.add_ball_and_stick('not protein')
view